## Request Página de Busca

In [50]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import random
import time

#Passo a URL e coleto a lista de vagas de cientista de dados no Brasil
headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36 Edg/129.0.0.0"}
lista_soups = []
lista_cargos = ['cientista-de-dados', 'analista-de-dados', 'engenheiro-de-dados', 'analista-bi']

for cargo in lista_cargos:
   for page in range(1, 11):
      intervalo = random.uniform(2, 5)
      time.sleep(intervalo)

      url = f'https://www.catho.com.br/vagas/{cargo}/?page={page}'

      response = requests.get(url, headers=headers)
      if response.status_code == 200:
         soup = BeautifulSoup(response.text, 'html.parser')
         lista_soups.append(soup)
      else:
         print(f'Erro no cargo {cargo}, página {page}: {response.status_code}')

In [51]:
#junto todas as listas em um único soup
soup = BeautifulSoup('', 'html.parser')
for s in lista_soups:
   soup.append(s)

## Raspagem dos links das vagas

In [54]:
#Obtenho todos os links das vagas

conteudo = soup.find_all('h2', {'class': 'Title-module__title___3S2cv'})
links = []
for c in conteudo:
    try:
        link = c.find('a').get('href')
        links.append(link)
    except: 
        pass
links

['https://www.catho.com.br/vagas/trabalhe-de-casa-cientista-de-dados-senior/30672086/',
 'https://www.catho.com.br/vagas/cientista-de-dados-sr-44568/30647410/',
 'https://www.catho.com.br/vagas/cientista-de-dados/30142353/',
 'https://www.catho.com.br/vagas/cientista-de-dados/30427409/',
 'https://www.catho.com.br/vagas/cientista-de-dados/30412206/',
 'https://www.catho.com.br/vagas/cientista-de-dados/29119549/',
 'https://www.catho.com.br/vagas/cientista-de-dados/30053866/',
 'https://www.catho.com.br/vagas/cientista-de-dados/30015621/',
 'https://www.catho.com.br/vagas/cientista-de-dados/30001357/',
 'https://www.catho.com.br/vagas/cientista-de-dados-senior/30612839/',
 'https://www.catho.com.br/vagas/cientista-de-dados-senior/30210624/',
 'https://www.catho.com.br/vagas/cientista-de-dados-senior-genia/30252819/',
 'https://www.catho.com.br/vagas/cientista-de-dados-pleno/30015874/',
 'https://www.catho.com.br/vagas/cientista-de-dados-pleno/30002711/',
 'https://www.catho.com.br/vagas

In [105]:
#Removo links duplicados
links = list(set(links))

#Quantidade de links obtidos
print(len(links))

#gravar a lista de links em um arquivo csv
path = r'C:\Users\RodrigoPintoMesquita\Documents\GitHub\PB_TP1\app\data\raw\links_vagas.csv'
df = pd.DataFrame(links)
df.to_csv(path, index=False, header=False)

420

## Iteração realizando Request em cada link, salvando o HTML de cada vaga

In [122]:
#leio do arquivo onde salvei a relação de links
path_lista_vagas = r'C:\Users\RodrigoPintoMesquita\Documents\GitHub\PB_TP1\app\data\raw\links_vagas.csv'
with open(path_lista_vagas, 'r') as file:
    links = file.read().splitlines()


# faço um Loop por cada URL e salvo o conteúdo em um arquivo html
headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36 Edg/129.0.0.0"}

for i, url in enumerate(links, start=0):
    html_file = f'C:/Users/RodrigoPintoMesquita/Documents/GitHub/PB_TP1/app/data/html_pages/page_{i}.html'

    intervalo = random.uniform(2, 5)
    time.sleep(intervalo)

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        html_content = response.text
        with open(html_file, 'w', encoding='utf-8') as file:
            file.write(html_content)
    else:
        print(f'Erro na iteração: {i} | Status: {response.status_code} | Motivo: {response.reason}')

Erro na iteração: 211 | Status: 502 | Motivo: Bad Gateway


## Iteração em cada arquivo Html para obter os dados necessários

In [30]:
import os
import pandas as pd
from bs4 import BeautifulSoup
import re

files = os.walk(r'C:\Users\RodrigoPintoMesquita\Documents\GitHub\PB_TP1\app\data\html_pages')
files = list(files)[0][2]
paths_files = [f'C:/Users/RodrigoPintoMesquita/Documents/GitHub/PB_TP1/app/data/html_pages/{file}' for file in files]
paths_files

['C:/Users/RodrigoPintoMesquita/Documents/GitHub/PB_TP1/app/data/html_pages/page_0.html',
 'C:/Users/RodrigoPintoMesquita/Documents/GitHub/PB_TP1/app/data/html_pages/page_1.html',
 'C:/Users/RodrigoPintoMesquita/Documents/GitHub/PB_TP1/app/data/html_pages/page_10.html',
 'C:/Users/RodrigoPintoMesquita/Documents/GitHub/PB_TP1/app/data/html_pages/page_100.html',
 'C:/Users/RodrigoPintoMesquita/Documents/GitHub/PB_TP1/app/data/html_pages/page_101.html',
 'C:/Users/RodrigoPintoMesquita/Documents/GitHub/PB_TP1/app/data/html_pages/page_102.html',
 'C:/Users/RodrigoPintoMesquita/Documents/GitHub/PB_TP1/app/data/html_pages/page_103.html',
 'C:/Users/RodrigoPintoMesquita/Documents/GitHub/PB_TP1/app/data/html_pages/page_104.html',
 'C:/Users/RodrigoPintoMesquita/Documents/GitHub/PB_TP1/app/data/html_pages/page_105.html',
 'C:/Users/RodrigoPintoMesquita/Documents/GitHub/PB_TP1/app/data/html_pages/page_106.html',
 'C:/Users/RodrigoPintoMesquita/Documents/GitHub/PB_TP1/app/data/html_pages/page_107.

In [57]:
df = pd.DataFrame(columns=['url','titulo', 'local', 'salario', 'regime', 'descricao'])


for row_file in paths_files:
    with open(row_file, 'r', encoding='utf-8') as file:
        html_content = file.read()
        soup_vaga = BeautifulSoup(html_content, 'html.parser')

        #Título da vaga
        titulo_vaga = soup_vaga.find('h1')
        if titulo_vaga: titulo_vaga = titulo_vaga.text
        else: titulo_vaga = "não disponível"

        #Local
        local_vaga = soup_vaga.find('div', {'class': 'cidades'})
        if local_vaga: local_vaga = local_vaga.get_text()
        else: local_vaga = "não disponível"

        #Salario
        salario_img = soup_vaga.find('img', {'alt': 'Um icone representando dinheiro'})
        if salario_img: salario_vaga = salario_img.find_parent('li').get_text()
        else: salario_vaga = "não disponível"

        #Regime
        h3_reg = soup_vaga.find('h3', string="Regime de Contratação")
        if h3_reg: regime = h3_reg.find_next('p').get_text() 
        else: regime = "não disponível"

        #Descrição da vaga
        descricao_vaga = soup_vaga.find('div', {'class': 'job-description'})
        if descricao_vaga: descricao_vaga = descricao_vaga.get_text()
        else: descricao_vaga = "não disponível"

        #url
        indice = int(re.search(r'page_(\d+)\.html', row_file).group(1))
        df_urls = pd.read_csv(path, header=None)
        lista_urls = df_urls[0].tolist()
        url = lista_urls[indice]
        
        #Appendar a nova linha ao DataFrame
        nova_linha = pd.DataFrame([{
            'url': url,
            'titulo': titulo_vaga,
            'local': local_vaga, 
            'salario': salario_vaga,
            'regime': regime, 
            'descricao': descricao_vaga}])

        df = pd.concat([df, nova_linha], ignore_index=True)

        #Salvar em CSV com aspas nos textos
        df.to_csv(r'C:\Users\RodrigoPintoMesquita\Documents\GitHub\PB_TP1\app\data\raw\vagas.csv', index=False, quoting=1)
        


In [58]:
df

,url,titulo,local,salario,regime,descricao
0,https://www.catho.com.br/vagas/analista-de-dad...,Vaga de Analista de Dados - Júnior,Ipatinga - MG (1),A combinar,não disponível,"\nPara agregar nosso time, suas principais ati..."
1,https://www.catho.com.br/vagas/cientista-de-da...,Vaga de Cientista de Dados Sênior,São Paulo - SP (1),A combinar,não disponível,Skills / Atividades: Conhecimentos em Conhecim...
2,https://www.catho.com.br/vagas/analista-de-dad...,Vaga de Analista de Dados em Excel,São Paulo - SP (1),A combinar,não disponível,Com experiência comprovada como Analista de Si...
3,https://www.catho.com.br/vagas/analista-de-dad...,Vaga de Analista de Dados,Brasilia - DF (1),A combinar,não disponível,-Analisa estratégias e atividades de concorren...
4,https://www.catho.com.br/vagas/analista-de-bi-...,Vaga de Analista de BI Sr,São Paulo - SP (1),"De R$ 4.001,00 a R$ 5.000,00",não disponível,Quer fazer parte de uma rede de reconhecimento...
...,...,...,...,...,...,...
414,https://www.catho.com.br/vagas/analista-de-dad...,Vaga de ANALISTA DE DADOS,São Paulo - SP (1),"Até R$ 2.000,00",não disponível,ANALISTA DE REPORTES E DADOS\n\nAtividades:\n-...
415,https://www.catho.com.br/vagas/analista-de-bi-...,Vaga de Analista de BI Sênior - Operação de Ca...,Barueri - SP (2),"De R$ 5.001,00 a R$ 6.000,00",não disponível,Realizará o envio de informações gerenciais po...
416,https://www.catho.com.br/vagas/analista-de-dad...,Vaga de Analista de Dados/BI,São Paulo - SP (1),A combinar,não disponível,Modelo de trabalho: Remoto + presencial 01x po...
417,https://www.catho.com.br/vagas/especialista-em...,Vaga de Especialista em Engenharia de Dados - ...,Curitiba - PR (1),A combinar,não disponível,Desenvolver e gerenciar pipelines de dados em ...


## Refatoração do código encapsulando em funções

In [2]:
%%writefile functions_webscrapping.py


import pandas as pd
import requests
from bs4 import BeautifulSoup
import random
import time
import os
import re
import json

def obter_paginas_de_busca(lista_cargos, range_max):
    """
    Através do Request, esta função obtém os resultados de pesquisa no site da Catho 
    para cada cargo passado na lista fornecida, e retorna um objeto BeautifulSoup
    contendo o conteúdo HTML de todas as páginas obtidas.

    Args:
        lista_cargos (list): Uma lista de strings representando os cargos a serem buscados.
        range_max (int): O número máximo de páginas a serem buscadas para cada cargo.

    Returns:
        BeautifulSoup: Um objeto BeautifulSoup contendo o HTML de todas as páginas obtidas.
    """

    headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36 Edg/129.0.0.0"}
    lista_soups = []

    for cargo in lista_cargos:
       for page in range(1, range_max):
          intervalo = random.uniform(2, 5)
          time.sleep(intervalo)

          url = f'https://www.catho.com.br/vagas/{cargo}/?page={page}'

          response = requests.get(url, headers=headers)
          if response.status_code == 200:
             soup = BeautifulSoup(response.text, 'html.parser')
             lista_soups.append(soup)
          else:
             print(f'Erro no cargo {cargo}, página {page}: {response.status_code}')

    soup_paginas_de_busca = BeautifulSoup('', 'html.parser')
    for s in lista_soups:
       soup_paginas_de_busca.append(s)
    return soup_paginas_de_busca

def raspas_paginas_e_salvar_links(soup_paginas_de_busca, links_path):
    """
    Esta função procura por links de vagas no objeto BeautifulSoup fornecido,
    extrai esses links e os salva em um arquivo CSV no caminho especificado.

    Args:
        soup_paginas_de_busca (BeautifulSoup): Objeto BeautifulSoup contendo o HTML das páginas de busca.
        links_path (str): Caminho do arquivo CSV onde os links serão salvos.

    Returns:
        list: Uma lista contendo todos os links extraídos.
    """
    conteudo = soup_paginas_de_busca.find_all('h2', {'class': 'Title-module__title___3S2cv'})
    links = []
    for c in conteudo:
        try:
            link = c.find('a').get('href')
            links.append(link)
        except: 
            pass

    df = pd.DataFrame(list(set(links)))
    df.to_csv(links_path, index=False, header=False)
    
    return links


def iterar_links_e_salvar_paginas_html(links_path, htmls_folder_path):
    """
    Esta função lê links de um arquivo CSV, faz requisições HTTP para cada link
    e salva o conteúdo HTML de cada página em arquivos separados no caminho fornecido.

    Args:
        links_path (str): Caminho do arquivo CSV contendo os links das vagas.
        htmls_folder_path (str): Caminho da pasta onde os arquivos HTML serão salvos.
    """

    with open(links_path, 'r') as file:
        links = file.read().splitlines()

    headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36 Edg/129.0.0.0"}

    for i, url in enumerate(links, start=0):
        html_file = f'{htmls_folder_path}/page_{i}.html'

        intervalo = random.uniform(2, 5)
        time.sleep(intervalo)

        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            html_content = response.text
            with open(html_file, 'w', encoding='utf-8') as file:
                file.write(html_content)
        else:
            print(f'Erro na iteração: {i} | Status: {response.status_code} | Motivo: {response.reason}')


def iterar_htmls_e_extrair_dados_deprecada(links_path, htmls_folder_path, path_csv_resultado):
    """
    Esta foi a primeira função desenvolvida para extrair dados dos arquivos HTML obtidos das vagas pesquisadas, 
    usando apenas seletores CSS. Porém consegui uma forma melhor pegando do elemento "Props" da página e depois fazendo
    a leitura como um Json.
    """

    files = os.walk(htmls_folder_path)
    files = list(files)[0][2]
    paths_files = [f'{htmls_folder_path}/{file}' for file in files]

    df = pd.DataFrame(columns=['url','titulo', 'local', 'salario', 'regime', 'descricao'])
    df_urls = pd.read_csv(links_path, header=None)
    lista_urls = df_urls[0].tolist()

    for row_file in paths_files:
        with open(row_file, 'r', encoding='utf-8') as file:
            html_content = file.read()
            indice = int(re.search(r'page_(\d+)\.html', row_file).group(1))
            url = lista_urls[indice]
            
        soup_vaga = BeautifulSoup(html_content, 'html.parser')

        titulo_vaga = soup_vaga.find('h1')
        titulo_vaga = titulo_vaga.text if titulo_vaga else "não disponível"

        local_vaga = soup_vaga.find('div', {'class': 'cidades'})
        local_vaga = local_vaga.get_text() if local_vaga else "não disponível"

        salario_img = soup_vaga.find('img', {'alt': 'Um icone representando dinheiro'})
        salario_vaga = salario_img.find_parent('li').get_text() if salario_img else "não disponível"

        h3_reg = soup_vaga.find('h3', string="Regime de Contratação")
        regime = h3_reg.find_next('p').get_text() if h3_reg else "não disponível"

        descricao_vaga = soup_vaga.find('div', {'class': 'job-description'})
        descricao_vaga = descricao_vaga.get_text() if descricao_vaga else "não disponível"

        nova_linha = pd.DataFrame([{
            'url': url,
            'titulo': titulo_vaga,
            'local': local_vaga, 
            'salario': salario_vaga,
            'regime': regime, 
            'descricao': descricao_vaga
        }])

        df = pd.concat([df, nova_linha], ignore_index=True)

        df.to_csv(path_csv_resultado, index=False, quoting=1)

    return df


def iterar_htmls_e_extrair_dados(links_path, htmls_folder_path, path_csv_resultado):
    """
        --------------------------------NOVA FUNÇÃO------------------------
        Esta função lê todos os arquivos HTML obtidos das vagas pesquisadas, extrai informações como título,
        local, salário, regime e descrição, e salva em um arquivo csv

        Args:
            links_path (str): Caminho do arquivo CSV contendo os links das vagas.
            htmls_folder_path (str): Caminho da pasta contendo os arquivos HTML das vagas.

        Returns:
            pandas.DataFrame: Um DataFrame contendo as informações extraídas das vagas.

        Note:
            O DataFrame retornado contém as colunas: 'url', 'titulo', 'local', 'salario',
            'regime' e 'descricao'.
    """

    files = os.walk(htmls_folder_path)
    files = list(files)[0][2]
    paths_files = [f'{htmls_folder_path}/{file}' for file in files]

    df = pd.DataFrame(columns=['id_vaga', 'data_anuncio', 'titulo_vaga', 'titulo_resumo', 'faixa_salarial', 'empresa_contratante', 'estado', 'cidade', 'url', 'descricao', 'beneficios', 'regimeContrato'])

    for row_file in paths_files:
        with open(row_file, 'r', encoding='utf-8') as file:
            html_content = file.read()

            try:
                soup = BeautifulSoup(html_content, 'html.parser')

                script = soup.find('script', id='__NEXT_DATA__')

                json_data = json.loads(script.string)
                json_data_job = json_data['props']['pageProps']['jobAdData']

                id_vaga = json_data_job.get('id', "não disponível")
                data_anuncio = json_data_job.get('data', "não disponível")
                titulo_vaga = json_data_job.get('titulo', "não disponível")
                titulo_resumo = json_data.get('query', {}).get('slug', "não disponível")[0]
                faixa_salarial = json_data_job.get('faixaSalarial', "não disponível")
                empresa_contratante = json_data_job.get('contratante', {}).get('nome', "não disponível")
                estado = json_data_job.get('vagas', [{}])[0].get('uf', "não disponível")
                cidade = json_data_job.get('vagas', [{}])[0].get('cidade', "não disponível")
                url = json_data['props']['pageProps']['pathname']
                descricao = json_data_job.get('descricao', "não disponível")
                descricao = descricao.replace('\n', '')
                beneficios = json_data_job.get('benef', "não disponível")
                regime_contrato = json_data_job.get('regimeContrato', "não disponível")


                nova_linha = pd.DataFrame([{
                    "id_vaga": id_vaga,
                    "data_anuncio": data_anuncio,
                    "titulo_vaga": titulo_vaga,
                    "titulo_resumo": titulo_resumo,
                    "faixa_salarial": faixa_salarial,
                    "empresa_contratante": empresa_contratante,
                    "estado": estado,
                    "cidade": cidade,
                    "url": url,
                    "descricao": descricao,
                    "beneficios": beneficios,
                    "regimeContrato": regime_contrato
                }])

                df = pd.concat([df, nova_linha], ignore_index=True)

            except:
                print(f'Erro na iteração: {row_file}')
            
        df.to_csv(path_csv_resultado, index=False, quoting=1, encoding='utf-8')

    return df

Writing functions_webscrapping.py


In [1]:
%%writefile main.py'

from app.services.functions_webscrapping import *

def main():
    lista_cargos = ['cientista-de-dados', 'analista-de-dados', 'engenheiro-de-dados', 'analista-bi']
    links_path = r'C:\Users\RodrigoPintoMesquita\Documents\GitHub\PB_TP1\app\data\raw\links_vagas.csv'
    htmls_folder_path = r'C:\Users\RodrigoPintoMesquita\Documents\GitHub\PB_TP1\app\data\html_pages'
    path_csv_resultado = r'C:\Users\RodrigoPintoMesquita\Documents\GitHub\PB_TP1\app\data\processed\vagas.csv'

    soup_paginas_de_busca = obter_paginas_de_busca(lista_cargos, 11)
    print(soup_paginas_de_busca)
    
    links = raspas_paginas_e_salvar_links(soup_paginas_de_busca, links_path)
    print(links)

    iterar_links_e_salvar_paginas_html(links_path, htmls_folder_path)

    df_final = iterar_htmls_e_extrair_dados(links_path, htmls_folder_path, path_csv_resultado)

    #Conferir a quantidade de parses com sucesso
    print(df_final.shape)

if __name__ == "__main__":
    main()

Writing app/services/main.py


FileNotFoundError: [Errno 2] No such file or directory: 'app/services/main.py'